In [123]:
import pandas as pd
from razdel import tokenize as razdel_tokenize
from nltk.stem.snowball import SnowballStemmer
import pymorphy2
import numpy as np
import re
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import RegexpTokenizer
from nltk import PorterStemmer
import pickle
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab

In [124]:
data = pd.read_csv(r'C:\Users\marik\afile\123\1\extremism_dataset.csv',on_bad_lines='skip')

In [125]:
labelencoder = LabelEncoder()
data['extremist_type_encoded'] = labelencoder.fit_transform(data['extremist_type'])
data[['extremist_type', 'extremist_type_encoded']].value_counts()

extremist_type  extremist_type_encoded
ethnicity       0                         6000
non_extremist   1                         6000
political       2                         6000
religious       3                         6000
Name: count, dtype: int64

#### Preprocessing

In [126]:
# Функция перед перед
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()
# Уменьшаем текст
def text_lower(text):
    return text.lower()


def clean_stopwords(text):

    stopwordlist = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как',
                    'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к',
                    'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне',
                    'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему',
                    'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже',
                    'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь',
                    'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего',
                    'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней',
                    'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб',
                    'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет',
                    'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого','это',
                    'эта','мы','они','для','с','наш','кто','от','тех','нам','от','наша',
                    'нашей','наша','своя','свои','своих','должны','должен','стоит','нашими',
                    '-нашей','эти','этой','нас','"','!']
    STOPWORDS = set(stopwordlist)
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

# Убираем пунктуацию 
def clean_puctuations(text):
    english_puctuations = string.punctuation
    translator = str.maketrans('','', english_puctuations)
    return text.translate(translator)

# Повторяющиеся знаки
def clean_repeating_characters(text):
    return re.sub(r'(.)1+', r'1', text)

# Ссылки тоже чистим
def clean_URLs(text):
    return re.sub(r"((www.[^s]+)|(http\S+))","",text)

# Удаляем все цифры
def clean_numeric(text):
    return re.sub('[0-9]+', '', text)

# Разбиваем текст
def tokenize_tweet(text):
    return [token.text for token in razdel_tokenize(text)]

# Убираем окончания  
def text_stemming(words):
    stemmer = SnowballStemmer("russian")
    return [stemmer.stem(word) for word in words]

# превращаем в неизмененные слова
def text_lemmatization(text):
    doc = Doc(text)
    doc.segment(segmenter)  # вот это важно!
    doc.tag_morph(morph_tagger)
    
    lemmas = []
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemmas.append(token.lemma)
    return lemmas

In [127]:
def preprocess(text_series):
    text_series = text_series.apply(clean_URLs)
    text_series = text_series.apply(clean_numeric)
    text_series = text_series.apply(clean_puctuations)
    text_series = text_series.apply(text_lower)
    text_series = text_series.apply(clean_stopwords)
    text_series = text_series.apply(tokenize_tweet)
    text_series = text_series.apply(text_stemming)
    text_series = text_series.apply(lambda x: " ".join(x)) 
    text_series = text_series.apply(text_lemmatization)
    text_series = text_series.apply(lambda x: " ".join(x)) 
    return text_series
    


In [128]:
# Делим на тест и тренировку на 30 и 70
X, y = data['text'], data['extremist_type_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 41)

In [129]:
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features= 500000)
vectoriser.fit(X_train)
# print("No. of feature words: ",len(vectoriser.get_feature_names()))

TfidfVectorizer(max_features=500000, ngram_range=(1, 2))

In [130]:
# Берем вектора
pickle.dump(vectoriser, open(r"C:\Users\marik\OneDrive\Desktop\AE\tdf_vectorizer", 'wb'))

In [131]:
X_train = vectoriser.transform(X_train)
X_test = vectoriser.transform(X_test)

In [132]:
svm_model_linear = SVC(kernel= 'linear', C = 1).fit(X_train, y_train)
svm_predictions  = svm_model_linear.predict(X_test)
accuracy = svm_model_linear.score(X_test, y_test)
print(accuracy)

1.0


In [133]:
pickle.dump(svm_model_linear, open(r"C:\Users\marik\OneDrive\Desktop\AE\model.bin", 'wb'))

In [134]:
def custom_input_prediction(text):
    import nltk
    import pandas as pd
    import pickle

    nltk.download('omw-1.4')

    try:
        text_series = pd.Series(text)
        text_series = preprocess(text_series) 

        if text_series[0] is None or text_series[0].strip() == "":
            raise ValueError("Преобразованный текст пуст или недопустим")

        text = [text_series[0]]  

        with open(r"C:\Users\marik\OneDrive\Desktop\AE\tdf_vectorizer", "rb") as vec_file:
            vectoriser = pickle.load(vec_file)


        text_vector = vectoriser.transform(text)


        with open(r"C:\Users\marik\OneDrive\Desktop\AE\model.bin", "rb") as model_file:
            model = pickle.load(model_file)


        prediction = model.predict(text_vector)[0] 


        interpretations = {
            0: "ethnicity",
            1: "extremist",
            2: "non_extremist",
            3: "political",
            4: "religious"
        }
        return interpretations.get(prediction, "Unknown")

    except Exception as e:
        print(f"Произошла ошибка при анализе текста: {e}")
        return "Error"


In [137]:
example_1 = "Эта гендерно-нейтральная чушь разрушает наше общество! Моим внукам промывают мозги с младшей школы, а в старшей — полное безумие! Учителя играют в телефонах, пока девочек в 'унисекс' туалетах доводят до слёз! Если бы мой внук не вмешался, никто бы и не помог! Всё ради толерантности, плевать на безопасность!"
example_2 = "Когда убивают кашмирских пандитов, когда исламские фанатики насилуют и убивают — все молчат! Но стоит кому-то критиковать ислам — начинается истерика про 'исламофобию'! Двойные стандарты! Никто не осмеливается говорить правду про радикальный ислам, все боятся!"
example_3 = "Раньше можно было говорить правду — шутки про геев, изнасилования, всё было нормально, и никто не ныл! Сейчас вся культура прогнила, каждый обиженный требует извинений! Попробуй включи старые шоу — они были честнее и смелее, чем весь этот современный либеральный мусор!"
print(custom_input_prediction(example_1))

political


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\marik\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
